# RAG - Retrieval Augmented Generation

The purpose of a RAG system is to generate a response that is relevant to the user's query. The system should be able to retrieve relevant information from a knowledge base and use it to generate a response. This notebook will demonstrate how to use the RAG model to generate responses to user queries.

## Requirements

Before we start, make sure we have an embedding model to use for the RAG system.
for this tutorial we will be using the `nomic-embed-text` model.

in your terminal run the following command:

```bash
ollama pull nomic-embed-text
```

verify that the model is downloaded by running the following command:

```bash
ollama models
```


First we will need a document to work with. We will be using the a code of conduct as our document.
Options for tools to load documents include:
- `PyPDFLoader`
- `WebBaseLoader`
- `ObsidianLoader` 
- `RedditPostsLoader`
- `RecursiveUrlLoader`
- `TextLoader`

In [1]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt"
]
loader = WebBaseLoader(urls)
documents = loader.load()
print(documents[0].page_content[:100])


USER_AGENT environment variable not set, consider setting it to identify your requests.


1.	Code of Conduct

Our Code of Conduct outlines the fundamental principles and ethical standards th


# Documents
Loaders will return a list of documents. Each document is a dictionary with the following keys:
- `page_content`: the text of the document
- `metadata`: a dictionary with additional information about the document
The document object will be the most important object in the RAG pipeline. It is used to create the embeddings of the documents.

# Chunking
after loading in our documents, we can now chunk them into smaller documents. This is done by using the `chunk` function from the `langchain.text_splitter` module. We can specify the chunk size and the chunk overlap. More can be found under `langchain_classic.text_splitter`

notable splitters are:
- `RecursiveCharacterTextSplitter`
- `MarkdownTextSplitter`
- `TextSplitter`
- `CharacterTextSplitter`

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, separators=["\n\n+(\d+\.)\s*"], is_separator_regex=True, keep_separator=False, strip_whitespace=True
)

chunks = splitter.split_documents(documents=documents)
print(chunks[1].page_content)

.	Recruitment Policy

Our Recruitment Policy reflects our commitment to attracting, selecting, and onboarding the most qualified and diverse candidates to join our organization. We believe that the success of our company relies on the talents, skills, and dedication of our employees.
Equal Opportunity: We are an equal opportunity employer and do not discriminate on the basis of race, color, religion, sex, sexual orientation, gender identity, national origin, age, disability, or any other protected status. We actively promote diversity and inclusion.
Transparency: We maintain transparency in our recruitment processes. All job vacancies are advertised internally and externally when appropriate. Job descriptions and requirements are clear and accurately represent the role.
Selection Criteria: Our selection process is based on the qualifications, experience, and skills necessary for the position. Interviews and assessments are conducted objectively, and decisions are made without bias.
Dat

# Embeddings Model
before we start utilizing our vector storage, we need an embeddings model to convert our text into a vector representation. We will use the Ollama `nomic-embed-text` model from ollama to do this.

other models from ollama also include: 
- `qwen3-embedding`
- `embeddinggemma`
- `snowflake-arctic-embed`

OpenAI also has its own embeddings model that can be used via the openai api key.
more details can be found here: https://docs.langchain.com/oss/python/integrations/text_embedding/openai

In [3]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Vector Database
with our documents chunked and our embeddings model ready, we can now create a vector database. We will use the `ChromaDB` library for this. 

The vector database can be used to store the embeddings locally or it cached at run time. We will use the cached version for this example.
if you'd like to have a persistant vector database you can change the persistant_directory to a folder


In [4]:
from langchain_chroma import Chroma
vectorstore = Chroma(
    collection_name="elden_ring_docs",
    embedding_function=embeddings,
    persist_directory=None  # This makes it persistent
)

# Adding Documents to the Knowledge Base

In [5]:
vectorstore.add_documents(chunks)

['321d11f4-db96-4ecc-9f85-9aa1f5452bd2',
 '45ca064a-ea72-4ed1-ae0e-5462141f43ff',
 '74575687-32b4-4a04-a3f7-1f94f73b3e15',
 '25f27b13-66be-4f9f-aa4a-910a1487c382',
 'f7e5a627-a5e9-4abd-84fb-dc3f9fbd79cc',
 'a283fd80-e1c6-4865-a0c9-0a1e510de07c',
 '01c6c4b4-4d3c-4060-ab76-a5eee055f484',
 '7232a311-406f-4844-8f75-9fe92694ba5b',
 'eb18abaf-0ff3-4cd4-9238-7e257c5d456c',
 '86661007-7731-4f71-ae7a-adf51f208483',
 '816bcf02-736f-42a1-8f43-265506289798',
 '46e79724-f63f-4de1-9a87-079cf3731ce2',
 '0a1b56a5-31b6-409e-8b91-379a57c4d672',
 '604b0ce0-fbdd-4e64-abba-7980ba1bf619',
 'a27b2103-ac63-4f76-8a72-fd57db9e89a9',
 '26895971-822e-4b53-b8fc-433482688df1',
 '5a19e862-a0d5-42ba-8552-49cb8a11bd90',
 '97ad5357-1cf7-43d2-853f-8fb4fd27812f',
 'ed078363-93ce-4f93-b4a2-e2288df953bb']

In [6]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

results = retriever.invoke("Recruitment Policy")

for result in results:
    print("-"*100)
    print("page content:\n", result.page_content)
    print("metadata:", result.metadata)

----------------------------------------------------------------------------------------------------
page content:
 Acceptable Use: Mobile devices are primarily intended for work-related tasks. Limited personal usage is allowed, provided it does not disrupt work obligations.
Security: Safeguard your mobile device and access credentials. Exercise caution when downloading apps or clicking links from unfamiliar sources. Promptly report security concerns or suspicious activities related to your mobile device.
Confidentiality: Avoid transmitting sensitive company information via unsecured messaging apps or emails. Be discreet when discussing company matters in public spaces.
Cost Management: Keep personal phone usage separate from company accounts and reimburse the company for any personal charges on company-issued phones.
Compliance: Adhere to all pertinent laws and regulations concerning mobile phone usage, including those related to data protection and privacy.
Lost or Stolen Devices: Im